# 동영상 뽑다가 마지막에 실패

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&sq=&o=&q=%EC%98%81%ED%99%94'
data = requests.get(url)
soup = BeautifulSoup(data.text, 'html.parser')
detail_url_list=[]

for i in range(20):
    detail_url_list.append('https://search.daum.net/search' + soup.find_all('div', class_='wrap_thumb')[i].find(class_='thumb')['href'])


In [3]:
movie_url = detail_url_list[1]
data = requests.get(movie_url)
soup = BeautifulSoup(data.text, 'html.parser')

In [4]:
movie_detail_url = soup.find_all('c-thumb')[1]['data-href']
datas = requests.get(movie_detail_url)
soup = BeautifulSoup(datas.text, 'html.parser')

In [103]:
soup.find('iframe')

In [ ]:
name_list = []
code_list = []
url = "https://www.megabox.co.kr/on/oh/oha/Movie/selectMovieList.do"
headers = {"User-Agent": "Mozilla/5.0"}

for page_number in range(1, 3):
    payload = {
        "currentPage": str(page_number),
        "recordCountPerPage": "20",
        "pageType": "ticketing",
        "ibxMovieNmSearch": "",
        "onairYn": "Y",
        "specialType": "",
        "specialYn": "N"}

    response = requests.post(url, json=payload, headers=headers)
    data = json.loads(response.text)

    for movie in data['movieList']:
        name_list.append(movie['movieNm'])
        code_list.append(movie['movieNo'])
        
        
url = 'https://www.megabox.co.kr/on/oh/oha/Movie/selectMovieStilDetail.do'
headers = {"User-Agent": "Mozilla/5.0"}

mp4_list = []
for code in code_list:
    payload = {
        'rpstMovieNo' : code
    }

    response = requests.post(url, json=payload, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        mp4 = soup.find('div', class_='swiper-slide').find('source')['src']
        mp4_list.append(mp4)
        
    except:
        mp4_list.append(0)

for a,b in zip(name_list, mp4_list):

In [38]:
import json

url = 'https://www.megabox.co.kr/on/oh/oha/Movie/selectMovieStilDetail.do'
headers = {"User-Agent": "Mozilla/5.0"}

mp4_list = []
for code in code_list:
    payload = {
        'rpstMovieNo' : code
    }

    response = requests.post(url, json=payload, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        mp4 = soup.find('div', class_='swiper-slide').find('source')['src']
        mp4_list.append(mp4)
        
    except:
        mp4_list.append(0)


In [53]:
for a,b in zip(name_list, mp4_list):
    

데드맨 https://s3550.smartucc.kr/encodeFile/3550/2024/01/10/797dbe7a8616db752bcfb5de3ab7321a_W.mp4
웡카 https://s3550.smartucc.kr/encodeFile/3550/2024/01/03/07f287e89dfa43ebe45a665b87cc9a7a_W.mp4
도그데이즈 https://s3550.smartucc.kr/encodeFile/3550/2024/01/10/7b1adaaff1c00f08c8dc7a3c401b1d75_W.mp4
아기상어 극장판: 사이렌 스톤의 비밀 https://s3550.smartucc.kr/encodeFile/3550/2024/01/17/0f580e8eef4f45e3a2ff89f7db122c4a_W.mp4
소풍 https://s3550.smartucc.kr/encodeFile/3550/2024/01/16/4f6ba4ef205f4bb44c45fce2a2bc1187_W.mp4
아가일 https://s3550.smartucc.kr/encodeFile/3550/2024/01/12/6d6037d4855ba2ea8a470a08e65297f4_W.mp4
아네모네 https://s3550.smartucc.kr/encodeFile/3550/2024/01/17/28960a1541198c939bd83ea9728cf8f6_W.mp4
건국전쟁 0
시민덕희 https://s3550.smartucc.kr/encodeFile/3550/2024/01/26/1af98fe4ebbea8b4eb961b60e0af8189_W.mp4
탑건: 매버릭 https://s3550.smartucc.kr/encodeFile/3550/2022/05/04/a2bb4362179ac0848a9bd6b23009077a_W.mp4
더 퍼스트 슬램덩크 https://s3550.smartucc.kr/encodeFile/3550/2022/12/19/e38bc1ca7b3d35a056cda5c43e7abb1f_W.mp4
청춘 

In [42]:
df = pd.DataFrame()
df['name'] = name_list
df['mp4'] = mp4_list

In [54]:
a = ['데드맨', '웡카']
b = [0, 0]

In [55]:
for a, b in zip(a,b):
    df.loc[df['name'] == a, 'mp4'] = b
    
df.head()

,name,mp4
0,데드맨,0
1,웡카,0
2,도그데이즈,https://s3550.smartucc.kr/encodeFile/3550/2024...
3,아기상어 극장판: 사이렌 스톤의 비밀,https://s3550.smartucc.kr/encodeFile/3550/2024...
4,소풍,https://s3550.smartucc.kr/encodeFile/3550/2024...


In [45]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
collection = client.movies.info
name_list_2 = []
for i in list(collection.find()):
    name_list_2.append(i['name'])

name_list_2

['웡카',
 '2023 AFC 카타르 아시안컵 4강 - 대한민국 vs 요르단',
 '건국전쟁',
 '시민덕희',
 '정글번치-월드투어',
 '추락의 해부',
 '비욘드 유토피아',
 '외계+인 2부',
 '괴물',
 '서울의 봄',
 '너의 이름은.',
 '크레센도 반 클라이번 콩쿠르 실황',
 '나의 올드 오크',
 '지오디 마스터피스 더 무비',
 '상견니',
 '말하고 싶은 비밀',
 '킹덤1',
 '킹덤2',
 '킹덤-엑소더스']

In [49]:
df2 = df[df['name'].isin(name_list_2)]
df2[df2['mp4'] != 0]

,name,mp4
1,웡카,https://s3550.smartucc.kr/encodeFile/3550/2024...
8,시민덕희,https://s3550.smartucc.kr/encodeFile/3550/2024...
12,추락의 해부,https://s3550.smartucc.kr/encodeFile/3550/2024...
19,괴물,https://s3550.smartucc.kr/encodeFile/3550/2023...
20,서울의 봄,https://s3550.smartucc.kr/encodeFile/3550/2023...
25,외계+인 2부,https://s3550.smartucc.kr/encodeFile/3550/2023...
31,나의 올드 오크,https://s3550.smartucc.kr/encodeFile/3550/2023...


In [95]:
from pymongo import MongoClient
import numpy as np
client = MongoClient('localhost', 27017)
db = client.movies

# cgv
db_dict = list(db['cgv'].find())
cgv = pd.DataFrame(db_dict)
cgv.drop(columns='_id', inplace=True)

# megabox
db_dict = list(db['megabox'].find())
megabox = pd.DataFrame(db_dict)
megabox.drop(columns='_id', inplace=True)

# naver
db_dict = list(db['naver'].find())
naver = pd.DataFrame(db_dict)
naver.drop(columns='_id', inplace=True)


# 다음 기준으로 제목 동일한 데이터만 추출                    
column = list(cgv.name.values)

naver = naver[naver['name'].isin(column)]
megabox = megabox[megabox['name'].isin(column)]

# merge
def merge(df1, df2):
    merge_df = pd.merge(df1, df2, on='name', suffixes=('_df1', '_df2'), how='left')
    merge_df['review_df2'] = merge_df['review_df2'].fillna(0)
    merge_df['review_df2'] = merge_df['review_df2'].apply(lambda x: [] if x==0 else x)
    merge_df['review'] = merge_df['review_df1'] + merge_df['review_df2']
    merge_df = merge_df.drop(['review_df1', 'review_df2'], axis=1)
    
    return merge_df

result2 = merge(cgv, megabox)
result = merge(result2, naver)

In [79]:
img_list = []
url_list = []
name_list = []
rank_list = []
date_list = []
runtime_list = []
country_list = []
people_list = []


data = requests.get('http://www.cgv.co.kr/movies/?lt=1&ft=1')
soup = BeautifulSoup(data.text, 'html.parser')

num = 19
for i in range(num):
    soup_filter_top = soup.find_all('div', class_='box-image')[i]

    img_list.append(soup_filter_top.find('img')['src'])
    url_list.append('http://www.cgv.co.kr' + soup_filter_top.find('a')['href'])

for url in url_list:
    data = requests.get(url)
    soup = BeautifulSoup(data.text, 'html.parser')

    name = soup.find(class_='title').find('strong').text
    name = ' '.join(name.split(', '))
    name_list.append(name)
    
    soup_filter = soup.find('div', class_='spec').find_all('dd')
    temp = soup_filter[-1].text
    numbers = ''.join(re.findall(r'\d', temp))
    
    if len(numbers) == 8:
        date_list.append(soup_filter[-1].text)
        
        text = soup_filter[-2].text
        runtime_list.append(text.split(',')[1].strip())
        country_list.append(text.split(',')[2].strip())
    else:
        date_list.append(soup_filter[-2].text)
        
        text = soup_filter[-3].text
        runtime_list.append(text.split(',')[1].strip())
        country_list.append(text.split(',')[2].strip())

    rank_list.append(soup.find('div', class_='egg-gage small').find(class_="percent").text)
    people_list.append(int(''.join(soup.find('p', class_='desc').find('em').text.split(','))))

movie_data = {
        'name': name_list,
        'url': url_list,
        'img_url': img_list,
        'people' : people_list,
        'score': rank_list,
        'date' : date_list,
        'country' : country_list,
        'runtime' : runtime_list,
    }
    
movies_df = pd.DataFrame(movie_data, columns=['name', 'url', 'img_url', 'mp4_url', 'people', 'score','date','country', 'runtime'])

temp_name_list = []
temp_code_list = []
url = "https://www.megabox.co.kr/on/oh/oha/Movie/selectMovieList.do"
headers = {"User-Agent": "Mozilla/5.0"}

for page_number in range(1, 3):
    payload = {
        "currentPage": str(page_number),
        "recordCountPerPage": "20",
        "pageType": "ticketing",
        "ibxMovieNmSearch": "",
        "onairYn": "Y",
        "specialType": "",
        "specialYn": "N"}

    response = requests.post(url, json=payload, headers=headers)
    data = json.loads(response.text)

    for movie in data['movieList']:
        temp_name_list.append(movie['movieNm'])
        temp_code_list.append(movie['movieNo'])
        
        
url = 'https://www.megabox.co.kr/on/oh/oha/Movie/selectMovieStilDetail.do'
headers = {"User-Agent": "Mozilla/5.0"}

mp4_list = []
for code in temp_code_list:
    payload = {
        'rpstMovieNo' : code
    }

    response = requests.post(url, json=payload, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        mp4 = soup.find('div', class_='swiper-slide').find('source')['src']
        mp4_list.append(mp4)
        
    except:
        mp4_list.append('0')

df_temp = pd.DataFrame()
df_temp['name'] = temp_name_list
df_temp['mp4_url'] = mp4_list

df_t = pd.merge(movies_df, df_temp, on='name', how='left').drop(columns='mp4_url_x')
df_t.rename(columns={'mp4_url_y' : 'mp4_url'}, inplace=True)
df_t.mp4_url = df_t.mp4_url.apply(lambda x: '0' if pd.isna(x) else x)

df_t = df_t.to_dict(orient='records')
# # connection = pymysql.connect(
# #     host='movie-db.cte4qk2ucq5d.ap-northeast-2.rds.amazonaws.com',
# #     user='admin', 
# #     passwd='admin12345', 
# #     database='movie', 
# #     cursorclass=pymysql.cursors.DictCursor)

# # cursor = connection.cursor()
# # sql = "DELETE FROM movie_reviews"
# # cursor.execute(sql)
# # sql = "DELETE FROM movie_info"
# # cursor.execute(sql)

# # for index, row in movies_df.iterrows():
# #     sql = "INSERT INTO `df_t` (`name`, `url`, `img_url`, `people`, `score`, `country`, `runtime`) VALUES (%s, %s, %s, %s, %s, %s, %s)"
# #     cursor.execute(sql, (row['name'], row['url'], row['img_url'], row['people'], row['score'], row['country'], row['runtime']))

# # connection.commit()
    
    
    
client = MongoClient('localhost', 27017)
db = client.movies

# 삭제할 콜렉션 이름 리스트
collections_list = ['info']  # 삭제할 콜렉션 이름들을 여기에 추가
 
# 각 콜렉션을 하나씩 삭제
for collection_name in collections_list:
    db.drop_collection(collection_name)

collection = db.info
collection.insert_many(df_t)